In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

In [ ]:
from wall_analysis import parse_experiments
from crazyflie_description_py.experiments import WALL_ANGLE_DEG
import seaborn as sns
from simulation import get_df_theory
from plotting_tools import pcolorfast_custom
from plotting_tools import save_fig

linestyles = {"estimated": "-", "theo": ":", "theo_corr": "-.", "measured": "-"}
colors = {"estimated": "C0", "theo": "black", "theo_corr": "black", "measured":"C0"}
keys = {"estimated": "estimated", "theo": "theo, vertical", "theo_corr": "theoretical", "measured":"measured"}

MIN_Z_CM = 30 # minimum flying height

def plot_plositions(row, min_time=None, max_time=None, max_dist=None):
    positions_cm = row.positions[:, :3] * 100
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    mask_time = np.ones_like(row.seconds, dtype=np.bool)
    if min_time is not None:
        mask_time = (row.seconds > min_time) 
    else:
        mask_time = positions_cm[:, 2] > MIN_Z_CM
    if max_time is not None:
        mask_time = mask_time & (row.seconds < max_time)
    else:
        mask_time = positions_cm[:, 2] > MIN_Z_CM
        
    time = row.seconds[mask_time]
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[mask_time, 0], y=positions_cm[mask_time, 1], 
                    hue=time, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    axs[0].axis('equal')
    axs[0].legend(loc='lower right', title='time [s]')

    axs[1].plot(time, positions_cm[mask_time, 0], label='x')
    axs[1].plot(time, positions_cm[mask_time, 1], label='y')
    axs[1].plot(time, positions_cm[mask_time, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    if max_dist is not None:
        axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(time, row.positions[mask_time, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')
    plt.tight_layout()
    return fig, axs

def plot_audio(row, mic_idx=0):
    all_frequencies = row.freqs
    spec = row.spectrogram[:, mic_idx, :]
    spec[spec == 0] = np.nan
    total = np.nanmean(np.abs(spec), axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.set_title(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    
    # mark too long measurements gray
    max_diff = 1
    diff = row.seconds[1:] - row.seconds[:-1]
    indices = np.where(diff>max_diff)[0]
    endings = row.seconds[:-1][indices]
    diff_average = np.mean(diff[diff<max_diff])
    seconds = row.seconds
    for counter, i in enumerate(indices):
        new_time = seconds[i+counter]+diff_average
        seconds = np.insert(seconds, i+counter+1, new_time)
        spec = np.insert(spec, i+counter+1, np.nan, axis=1)
    
    pcolorfast_custom(ax, seconds, all_frequencies, np.abs(spec))
    
    xticks = np.arange(0, row.seconds.max(), step=5)
    ax.set_xticks(xticks); ax.set_xticklabels(xticks)
    yticks = np.arange((np.round(row.freqs.min()//1000)+1)*1000, 
                        row.freqs.max(), step=1000)
    ax.set_yticks(yticks); ax.set_yticklabels(yticks)
    ax.set_ylabel('frequency [Hz]')
    ax.set_xlabel('seconds [s]')
    return fig, ax

def plot_df(distance_range, freq_range=[2000, 6000], azimuth_deg=WALL_ANGLE_DEG, mic_idx=0):
    distances_grid = np.linspace(distance_range[0], distance_range[1],100)
    freqs_theo = np.linspace(freq_range[0], freq_range[1], 200)
    df_matrix_theo = get_df_theory(freqs_theo, distances_grid, azimuth_deg=azimuth_deg, 
                                   chosen_mics=[mic_idx])
    fig_df, ax_df = plt.subplots()
    fig_df.set_size_inches(5, 3)
    xticks = [7] + list(np.arange(10, distance_range[1]+1, step=10))
    yticks = np.arange(freq_range[0], freq_range[1]+1, step=1000)
    pcolorfast_custom(
        ax_df, distances_grid, freqs_theo, np.log10(df_matrix_theo[0]), cmap='Greys',
        alpha=0.5,
    )
    ax_df.set_xticks(xticks); ax_df.set_xticklabels(xticks)
    ax_df.set_yticks(yticks); ax_df.set_yticklabels(yticks)
    ax_df.set_xlabel('distance [cm]')
    ax_df.set_ylabel('frequency [Hz]')
    return fig_df, ax_df

def change_order(axs_all, mean_distances, xlabel="distance", unit="cm", ylabel="probability [-]", title=True):
    sorted_idx = np.argsort(mean_distances)
    positions = [ax.get_position() for ax in axs_all]
    for i, idx in enumerate(sorted_idx):
        axs_all[idx].set_position(positions[i])
        if title:
            axs_all[idx].set_title(f"{mean_distances[idx]:.0f}{unit}")
    [ax.get_yaxis().set_visible(False) for ax in axs_all[sorted_idx[1:]]]
    axs_all[sorted_idx[len(sorted_idx)//2]].set_xlabel(f"{xlabel} [{unit}]")
    axs_all[sorted_idx[0]].set_ylabel(ylabel)
    axs_all[sorted_idx[-1]].legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])
    return axs_all

# 1. Frequency slice

In [ ]:
exp_name = '2021_05_04_flying'; # snr = 3
#exp_name = '2021_07_14_flying_hover'; #  snr = 5
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

In [ ]:
df_total.sort_values(by='appendix', inplace=True)

## 1.1 positions analysis

In [ ]:
from geometry import Context
context = Context.get_crazyflie_setup(dim=2)
fig, ax = plt.subplots()
context.plot(ax=ax)
fig.set_size_inches(3, 3)

In [ ]:
min_time = None #4 #None
max_time = None #14 #None
max_dist = None

#starting_distance = 65.63 # 42+29.7−6.07 
starting_distance = 100

fig_total, ax_total = plt.subplots()
fig_total.set_size_inches(3, 3)
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time, max_time, max_dist)
    
    x = row.positions[:, 0] * 100
    y = row.positions[:, 1] * 100 - starting_distance
    z = row.positions[:, 2] * 100
    
    x = x[~np.isnan(x)] 
    y = y[~np.isnan(y)]
    
    x = x[z[~np.isnan(z)] > MIN_Z_CM]
    y = y[z[~np.isnan(z)] > MIN_Z_CM]
    
    #ax_total.scatter(x, y, s=10.0, label=row.appendix)
    ax_total.plot(x, y, marker='o', label=row.appendix.replace('_', ''))
    #mask2 = (y > -130) & (y < -100)
    #mask3 = (y > -100) & (y < -70)
    #mask4 = (y > -70)
    #for mask in [mask2, mask3, mask4]:
    #    ax_total.scatter(x[mask], y[mask], s=10.0)
    #save_fig(fig, f'plots/experiments/{exp_name}{row.appendix}_movement', extension='.png')
ax_total.axis('equal')
ax_total.set_xlabel('x [cm]')
ax_total.set_ylabel('y [cm]')
ax_total.axhline(0, color='k', label='wall')
ax_total.legend(bbox_to_anchor=[1.0, 1.0], loc='upper left')
#save_fig(fig_total, f'plots/experiments/{exp_name}_pos.png')

## 1.2 audio analysis

In [ ]:
from frequency_analysis import add_spectrogram

df_total = df_total.assign(spectrogram=None,freqs=None)
df_total = df_total.apply(add_spectrogram, axis=1)
print(df_total.columns)

mic_idx = 0

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    
    #complicated spectrogram
    fig, ax = plot_audio(row, mic_idx=mic_idx)
    #ax.set_ylim([2800, 5000])
    #save_fig(fig, f'plots/experiments/{exp_name}{row.appendix}_spec')
    
    #if not row.appendix in ["_bin5_thirdtry", "_bin6"]:
    #    continue
    #fig, ax = plt.subplots()
    #ax.pcolorfast(row.seconds, row.freqs, np.log10(np.abs(row.spectrogram[:-1, mic_idx, :-1])))
    #ax.set_title(row.appendix)

## 1.3 algorithm performance

In [ ]:
from calibration import get_calibration_function_median, get_calibration_function_dict
from inference import Inference

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
calib_function, calib_freq = get_calibration_function_median(
    "2021_04_30_stepper", "audio_deck", ax=ax, snr=3, motors=0#fit_one_gain=True 
)
inf_machine = Inference()
inf_machine.add_calibration_function(calib_function)

In [ ]:
from dataset_parameters import kwargs_datasets
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

kwargs = kwargs_datasets[exp_name]["audio_deck"]
azimuth_deg = WALL_ANGLE_DEG

distance_range = [7, 50]
freq_range = [3000, 5000]

fig_df, ax_df = plot_df(distance_range)

inf_machine.add_geometry(distance_range, azimuth_deg)

In [ ]:
# spec_masked, freqs_masked = data_collector.fill_from_row()
from copy import deepcopy
from data_collector import DataCollector
from estimators import DistanceEstimator, get_estimate
from inference import eps_normalize
from simulation import get_freq_slice_theory
from itertools import cycle

eps = 1e-5  # for plotting only
max_plot_distance = 110
algorithm = "bayes"
# algorithm = "cost"
normalize = True  # normalize probas before combining
method = "sum"  # method used to combine

nominal_ds = [60, 40, 20]

freqs_calib = np.linspace(
    np.min(row.frequencies_matrix), np.max(row.frequencies_matrix), 50
)
f_calib_all = calib_function(freqs_calib)

chosen_mics = None

plot_idx = 0
mean_distances = []

fig_df, ax_df = plot_df([7, max_plot_distance])
fig_all, axs_all = plt.subplots(1, 9)
fig_res, axs_res = plt.subplots(1, 9, sharex=True)
fig_all.set_size_inches(15, 3)
fig_res.set_size_inches(15, 2)

#row = df_total.loc[df_total.appendix == "_22"].iloc[0]
for i_row, row in df_total.iterrows():
    
    nominal_distances = cycle(nominal_ds)
    data_collector = DataCollector(exp_name=exp_name)
    print('treating', row.appendix)
    print('==================')
    
    flying_time_indices = list(np.where(row.positions[:, 2] * 1e2 > MIN_Z_CM)[0])

    count = 0
    sweep_complete = False

    for i in (flying_time_indices + [-1]):
        if i == row.stft.shape[0]:
            sweep_complete = True
            print("reached end of dataset")
        elif i == -1:
            sweep_complete = True
            print("reached end of dataset")
        elif row.bin_selection < 5:
            signals_f = row.stft[i]
            frequencies = row.frequencies_matrix[i]
            sweep_complete = data_collector.next_fslice_ready(
                signals_f, frequencies, verbose=False
            )
        elif i > 0:
            sweep_complete = True
        else:
            print("not ready yet")

        if sweep_complete:
            nominal_distance = next(nominal_distances)
            (
                f_slice,
                freqs,
                stds,
                distances,
            ) = data_collector.get_current_frequency_slice(verbose=False, df_cleanup=False)

            print("treating new frequency slice after", count)
            count = 0


            rel_distances = starting_distance - distances - nominal_distance
            inf_machine.add_data(
                deepcopy(f_slice), freqs, stds, deepcopy(rel_distances)
            )
            inf_machine.calibrate()

            # raw data, for plotting only
            freqs = inf_machine.values[inf_machine.valid_idx]
            distances = distances[inf_machine.valid_idx]

            distance_corr = starting_distance - distances
            mean_distance = np.nanmean(distance_corr)
            rel_distances = distance_corr - nominal_distance  # relative movement

            if mean_distance < max_plot_distance:
                ax_df.scatter(distance_corr, freqs, color=f"C{plot_idx}")
                ax_df.axvline(
                    mean_distance, color="black", ls=":", label="mean distance"
                )

            distance_estimators = {
                "measured": DistanceEstimator(),
                "theo": DistanceEstimator(),
                "theo_corr": DistanceEstimator(),
            }

            # treat measured data
            for i_mic in range(f_slice.shape[0]):
                dists, proba, diff = inf_machine.do_inference(
                    mic_idx=i_mic, algorithm=algorithm, normalize=normalize
                )
                distance_estimators["measured"].add_distribution(
                    diff * 1e-2, proba, i_mic
                )

            # treat theorectical data
            f_theo = get_freq_slice_theory(
                freqs, distance_cm=mean_distance, azimuth_deg=azimuth_deg
            ).T  # n_mics x n_freqs
            inf_machine.add_data(f_theo, freqs)
            for i_mic in range(f_theo.shape[0]):
                dists_theo, proba_theo, diff_theo = inf_machine.do_inference(
                    mic_idx=i_mic,
                    algorithm=algorithm,
                    normalize=normalize,
                    calibrate=False,
                )
                distance_estimators["theo"].add_distribution(
                    diff_theo * 1e-2, proba_theo, i_mic
                )

            f_theo_corr = get_freq_slice_theory(
                freqs, distance_cm=distance_corr, azimuth_deg=azimuth_deg
            ).T
            inf_machine.add_data(f_theo_corr, freqs, distances=rel_distances)
            for i_mic in range(f_theo_corr.shape[0]):
                (
                    dists_theo_corr,
                    proba_theo_corr,
                    diff_theo_corr,
                ) = inf_machine.do_inference(
                    mic_idx=i_mic,
                    algorithm=algorithm,
                    normalize=normalize,
                    calibrate=False,
                )
                distance_estimators["theo_corr"].add_distribution(
                    diff_theo_corr * 1e-2,
                    proba_theo_corr - np.mean(proba_theo_corr),
                    i_mic,
                )

            for key, distance_estimator in distance_estimators.items():

                distance_total, proba_total = distance_estimator.get_distance_distribution(method=method, chosen_mics=chosen_mics, azimuth_deg=azimuth_deg)


                if mean_distance < max_plot_distance:
                    axs_all[plot_idx].plot(
                        distance_total * 1e2,
                        eps_normalize(proba_total, eps),
                        label=keys[key],
                        ls=linestyles[key],
                        color=f"C{plot_idx}",
                    )

                    d = get_estimate(distance_total * 1e2, proba_total)
                    axs_res[plot_idx].axvline(
                        d, label=keys[key], ls=linestyles[key], color=f"C{plot_idx}",
                    )

            if mean_distance < max_plot_distance:
                axs_all[plot_idx].axvline(
                    mean_distance, color="k", ls=":", label="mean distance"
                )
                axs_all[plot_idx].set_yscale("log")

                axs_res[plot_idx].axvline(
                    mean_distance, color="k", ls=":", label="mean distance"
                )
                axs_res[plot_idx].scatter(distance_corr, freqs, color=f"C{plot_idx}")
                axs_res[plot_idx].set_ylim(3000, 5000)

                mean_distances.append(mean_distance)
                plot_idx += 1
            else:
                print('not plotting', mean_distance)

        if i < row.stft.shape[0]:
            signals_f = row.stft[i]
            frequencies = row.frequencies_matrix[i]
        
            mode = "maximum" if row.bin_selection < 5 else "all"
            data_collector.fill_from_signal(
                signals_f,
                frequencies,
                distance_cm=row.positions[i, 1] * 1e2,
                time=row.seconds[i],
                mode=mode,
            )
            count += 1
        else:
            print('done')


# sort subplots according to distance
axs_all = change_order(axs_all, mean_distances)
axs_res = change_order(axs_res, mean_distances, ylabel="frequency [Hz]", title=False)

#save_fig(fig_df, f"plots/experiments/{exp_name}_df.png")  # , extension="png")
#save_fig( fig_all, f"plots/experiments/{exp_name}_mics{chosen_mics}_all.png" )  # , extension="png")
#save_fig( fig_res, f"plots/experiments/{exp_name}_mics{chosen_mics}_res.png" )  # , extension="png")

# 2. Distance slice

In [ ]:
exp_name = '2021_05_04_linear';
starting_distance = 65.63 # 42+29.7−6.07 

# x, y, z, yaw
starting_poses = {
    '_1': [20, -starting_distance, 0, 45],
    '_2': [-20, -starting_distance, 0, -45],
    '_3': [0, -starting_distance, 0, 0],
    '_4': [-10, -starting_distance, 0, -30],
    '_5': [10, -starting_distance, 0, 30],
    '_fast1': [-20, -starting_distance, 0, -45],
    '_fast2': [20, -starting_distance, 0, 45],
    '_fast3': [0, -starting_distance, 0, 0],
    '_fast4': [20, -starting_distance, 0, 30],
    '_fast5': [-20, -starting_distance, 0, -30]
}
appendices = [f"_{i}" for i in range(1, 6)]#, "_fast2", "_fast4"]
n_rows = 3 
n_max = 14 # number of measurements to combine
step = 13 # plot every step slices 

In [ ]:
exp_name = '2021_10_12_linear';
starting_distance = 100 # 42+29.7−6.07 

# x, y, z, yaw
starting_poses = {
    '': [20, -starting_distance * np.cos(np.pi/4), 0, 45],
    '_1': [20, -starting_distance * np.cos(np.pi/4), 0, 45],
    '_2': [0, -starting_distance, 0, 0],
    '_6': [0, -starting_distance, 0, 0],
    '_8': [-20, -starting_distance * np.cos(np.pi/4), 0, -45],
    '_9': [-20, -starting_distance * np.cos(np.pi/4), 0, -45],
    '_10': [-20, -starting_distance * np.cos(np.pi/4), 0, -45],
}

appendices = ["", "_1", "_2", "_6", "_8", "_9"]#, "_fast2", "_fast4"]

n_rows = 8
n_max = 30 # number of measurements to combine
step = 10 # plot every step slices 

In [ ]:
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except Exception as e:
    print(e)
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

## 2.1 positions analysis

In [ ]:
def get_average_angle(positions_rot):
    # angles between -180, 180:
    angles = np.arctan2(positions_rot[:, 1]-positions_rot[0, 1], positions_rot[:, 0]-positions_rot[0, 0]) * 180 / np.pi
    
    # convert 120 to 60 etc.
    #angles[angles > 90] = 180 - angles[angles > 90]
    return np.median(angles)

def get_gt_angle(row):
    starting_yaw = starting_poses[row.appendix][3]
    approach_angle = starting_yaw + 90
    return approach_angle

def get_corrected_positions(appendix, positions):
    starting_pose = starting_poses[appendix]
    positions_rot = np.empty_like(positions)
    for j, pos in enumerate(positions):
        total_yaw = starting_pose[3] + pos[3]
        rot = R.from_euler('z', total_yaw, degrees=True)
        pos_rot = starting_pose[:3] + rot.apply(pos[:3]) * 1e2
        positions_rot[j, :] = np.r_[pos_rot, total_yaw]
    valid = np.all(~np.isnan(positions_rot), axis=1) & (positions_rot[:, 2] > 35)
    positions_rot = positions_rot[valid, :]
    return positions_rot

def plot_corrected_positions(row, max_idx=30, ax=None, **kwargs):
    if ax is None:
        fig, ax = plt.subplots()
        
    positions_rot = get_corrected_positions(row.appendix, row.positions)
    average_angle = get_average_angle(positions_rot)
    gt_angle = get_gt_angle(row)
    
    ax.plot(positions_rot[:max_idx, 0], positions_rot[:max_idx, 1], 
            label=f'experiment{row.appendix}, {average_angle:.0f} {gt_angle:.0f}', **kwargs)
    ax.axis('equal')
    return positions_rot

In [ ]:
from scipy.spatial.transform import Rotation as R

max_idx = 40
fig_df, ax_df = plot_df(distance_range=[7, 100])

fig, ax = plt.subplots()
fig.set_size_inches(3, 3)
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time=None, max_time=None, max_dist=None)
    plot_corrected_positions(row, ax=ax, max_idx=max_idx)
    
    positions_corr = get_corrected_positions(row.appendix, row.positions)
    ds = -positions_corr[:max_idx, 1]
    ax_df.scatter(ds, np.full(len(ds), 3000))
ax.legend(bbox_to_anchor=[1.0, 1.0], loc='upper left')

## 1.2 audio analysis

In [ ]:
from frequency_analysis import add_spectrogram
from plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0
#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    #fig, ax = plot_audio(row, mic_idx=mic_idx)
    
    plt.figure()
    for time_idx in range(row.frequencies_matrix.shape[0]):
        mic_idx = 0
        freqs = row.frequencies_matrix[time_idx, :]
        response = np.abs(row.spectrogram[:, mic_idx, time_idx])
        plt.plot(freqs, response)
        
    plt.title('experiment' + row.appendix)

## 1.3 algorithm analysis - old

In [ ]:
from inference import get_approach_angle_fft
from data_collector import DataCollector
from estimators import AngleEstimator, get_estimate
from copy import deepcopy

plot_raw = False #True #False

mics = [0, 1, 3]
#mics = [1, 3]
n_mics = len(mics)
#n_mics = 4


n_cols = len(appendices)

fig_all, axs_all = plt.subplots(n_rows, n_cols, sharey='row') # 10

[ax.set_xticks([]) for ax in axs_all.flatten()]
[ax.set_yticks([]) for ax in axs_all.flatten()]
axs_all[-1, n_cols//2].set_xlabel('angle [$^\\circ$]')
axs_all[n_rows//2, 0].set_ylabel('probability')
fig_all.set_size_inches(1.5*n_cols, 1.5*n_rows)

fig_res, axs_res = plt.subplots(n_rows, n_cols, sharey='row') # 10
[ax.set_xticks([]) for ax in axs_res.flatten()]
[ax.set_yticks([]) for ax in axs_res.flatten()]
axs_res[-1, n_cols//2].set_xlabel('x [cm]')
axs_res[n_rows//2, 0].set_ylabel('y [cm]')
fig_res.set_size_inches(1.5*n_cols, 1.5*n_rows)
    
#fig_df, ax_df = plot_df(distance_range=[7, 70], freq_range=[2000, 4000])


for k,  appendix in enumerate(appendices):
    row = df_total[df_total.appendix==appendix].iloc[0]
        
    slice_idx = 0
    fig_pos, ax_pos = plt.subplots()
    fig_pos.set_size_inches(3, 3)
    ax_pos.set_xlabel('x [cm]')
    ax_pos.set_ylabel('y [cm]')
    ax_pos.axis('equal')
        
    valid_positions = []
    
    angle_estimator = AngleEstimator()
    data_collector = DataCollector(exp_name=exp_name)
    
    angles = AngleEstimator.ANGLES_DEG
    results_matrix = np.zeros((len(angles), row.stft.shape[0]))
    
    for i in range(row.stft.shape[0]):
        signals_f = row.stft[i]
        frequencies = row.frequencies_matrix[i]
        position_cm = row.positions[i] * 1e2
        
        # need to start new dslice
        if data_collector.next_dslice_ready(signals_f, frequencies, position_cm):
            print('need to start new dslice')
            continue

        # only add measurements if the drone is flying.
        if data_collector.valid_dslice_measurement(
            position_cm, signals_f, frequencies, mono_freq=3000, verbose=False
        ):
            valid_positions.append(position_cm)
            #print('added', position_cm[1])
            data_collector.fill_from_signal(
                signals_f, frequencies, distance_cm=position_cm[1], time=row.seconds[i], 
                #mode="maximum"
                mode=3000
            )

        if (i % step) == 0:
            data =  data_collector.get_current_distance_slice(n_max=n_max)
            if data is not None:
                d_slices, rel_distances, stds, freqs = data
            else:
                #print(f'skipping step, cause no measurements yet')
                continue 
            if len(rel_distances) < n_max:
                #print(f'skipping step, cause only {len(rel_distances)} measurements')
                continue
            print(f'plotting up to {i} of {row.stft.shape[0]}')

            freq = np.mean(freqs)

            positions = np.array(valid_positions[-n_max:]) * 1e-2
            positions_corr = get_corrected_positions(row.appendix, positions)

            mean_angle = get_average_angle(positions_corr)

            ax_pos.scatter(positions_corr[:, 0], 
                           positions_corr[:, 1], 
                           color=f'C{slice_idx}',
                           s=10)
            ax_pos.set_title(f'experiment {k}')

            positions_here = positions_corr - positions_corr[0, :]
            
            axs_res[slice_idx, k].scatter(
                positions_here[:, 0], 
                positions_here[:, 1], 
                color=f'C{slice_idx}',
                s=10, 
                label="trajectory"
            )
            axs_res[0, k].set_title(f'exp. {k}')
            axs_all[0, k].set_title(f'exp. {k}')

            if plot_raw:
                fig, axs = plt.subplots(2, n_mics, sharey='row')
                fig.set_size_inches(10, 7)

            plot_mean_angle = mean_angle if mean_angle < 90 else 180 - mean_angle
            axs_all[slice_idx, k].axvline(plot_mean_angle, color='k', ls=':', label='mean angle')

            for i_mic, mic_idx in enumerate(mics):
                d_slice = d_slices[mic_idx]
                if plot_raw:
                    axs[0, i_mic].set_title(f'mic{mic_idx}')
                    ds = positions_corr[:, 1]
                    axs[0, i_mic].plot(ds, d_slice, color=f'C{slice_idx}', marker='o')
                    axs[0, i_mic].set_xlabel(f"relative distance [cm]")

                valid = ~np.isnan(d_slice)
                angles, proba = get_approach_angle_fft(
                    d_slice=d_slice,
                    frequency=freq,
                    relative_distances_cm=rel_distances,
                    reduced=True,
                    bayes=True
                )
                angle_estimator.add_distribution(angles, proba, mic_idx, freq)

                if plot_raw:
                    axs[1, i_mic].plot(angles, proba, color=f'C{slice_idx}')
                    axs[1, i_mic].axvline(plot_mean_angle, color='k', ls=':', label=mean_angle)
            if plot_raw:
                axs[0, 0].set_ylabel(f"magnitude{row.appendix}")
                axs[1, 0].set_ylabel(f"probability{row.appendix}")

            l=20
            axs_res[slice_idx, k].plot(
                [0, l*np.cos(mean_angle/180*np.pi)],
                [0, l*np.sin(mean_angle/180*np.pi)],
                color='k',
                ls=':',
                label='mean angle'
            )
            
            angles, probs = angle_estimator.get_angle_distribution(
                mics_left_right=[[1], [3]]
            )
            est_angle = get_estimate(angles, probs)
            
            results_matrix[:, i] = probs

            angles_plot = deepcopy(angles)
            angles_plot[angles_plot > 90] = 180 - angles[angles_plot > 90]
                
            axs_all[slice_idx, k].plot(angles_plot, probs, 
                                    color=f'C{slice_idx}', 
                                    label='estimated', 
                                    ls='-')
            #axs_all[slice_idx].set_yscale('log')

            axs_res[slice_idx, k].plot(
                [0, -l*np.cos(est_angle/180*np.pi)], 
                [0, l*np.sin(est_angle/180*np.pi)],
                color=f'C{slice_idx}',
                ls='-',
                label='estimates'
            )
            axs_res[slice_idx, k].axis('equal')

            slice_idx += 1
            if slice_idx > n_rows - 1: 
                break
                    
    ax_pos.axhline(0, color='k')
    #ax_pos.set_ylim(-100, 2)
    save_fig(fig_pos, f'plots/experiments/new_{exp_name}_pos{row.appendix}.png')
            
save_fig(fig_all, f'plots/experiments/new_{exp_name}_all.png')
save_fig(fig_res, f'plots/experiments/new_{exp_name}_res.png')
save_fig(fig_df,  f'plots/experiments/new_{exp_name}_df.png')

In [ ]:
from inference import get_approach_angle_fft
from data_collector import DataCollector
from estimators import AngleEstimator, get_estimate
from copy import deepcopy

mics = [0, 1, 2, 3]
n_mics = len(mics)
print('using n_max', n_max)

for k,  appendix in enumerate(appendices):
    print("appendix", appendix)
    row = df_total[df_total.appendix==appendix].iloc[0]
        
    data_collector = DataCollector(exp_name=exp_name)
    
    angles = AngleEstimator.ANGLES_DEG
    
    results_matrix = np.zeros((len(angles), row.stft.shape[0]))
    pos_matrix = np.zeros((4, row.stft.shape[0]))
    angle_vector = np.zeros(row.stft.shape[0])
    
    for i in range(row.stft.shape[0]):
        signals_f = row.stft[i]
        frequencies = row.frequencies_matrix[i]
        position_cm = row.positions[i] * 1e2
        
        # need to start new dslice
        #if data_collector.next_dslice_ready(signals_f, frequencies, position_cm):
        #    print('need to start new dslice')
        #    continue

        # only add measurements if the drone is flying.
        if data_collector.valid_dslice_measurement(
            position_cm, signals_f, frequencies, mono_freq=3000, verbose=False
        ):
            pos_matrix[:, i] = position_cm
            #print("fillig", position_cm[1], np.abs(signals_f[0, 15:19]), frequencies[15:19])
            data_collector.fill_from_signal(
                signals_f, frequencies, distance_cm=position_cm[1], time=row.seconds[i], 
                #mode="maximum"
                mode=3000
            )

        data =  data_collector.get_current_distance_slice(n_max=n_max)
        if data is not None:
            d_slices, rel_distances, *_ = data
        else:
            #print(f'skipping step, cause no measurements yet')
            continue 
        if len(rel_distances) < n_max:
            #print(f'skipping step, cause only {len(rel_distances)} measurements')
            continue

        freq = np.mean(freqs)

        positions = deepcopy(pos_matrix[:, i-n_max:i].T) * 1e-2
        positions_corr = get_corrected_positions(row.appendix, positions)
        mean_angle = get_average_angle(positions_corr)

        plot_mean_angle = mean_angle if mean_angle < 90 else 180 - mean_angle
        angle_vector[i] = plot_mean_angle

        angle_estimator = AngleEstimator()
        for i_mic, mic_idx in enumerate(mics):
            d_slice = d_slices[mic_idx]
            angles, proba = get_approach_angle_fft(
                d_slice=d_slice,
                frequency=freq,
                relative_distances_cm=rel_distances,
                reduced=True,
                bayes=True
            )
            angle_estimator.add_distribution(angles, proba, mic_idx, freq)

        angles, probs = angle_estimator.get_angle_distribution(
            mics_left_right=[[1], [3]]
        )
        est_angle = get_estimate(angles, probs)
        results_matrix[:, i] = probs
        
    distance_idx = np.arange(results_matrix.shape[1])
    gt_angle = starting_poses[row.appendix][3]
    gt_angle = 90 - gt_angle if gt_angle >= 0 else -gt_angle
    
    fig, axs = plt.subplots(1, 2)
    fig.set_size_inches(10, 3)
    axs[0].pcolorfast(distance_idx, angle_estimator.ANGLES_DEG, results_matrix)
    #axs[0].pcolorfast(distance_idx, angle_estimator.ANGLES_DEG, np.log10(results_matrix))
    axs[0].axhline(gt_angle, color='C1')
    axs[0].plot(distance_idx, angle_vector, color='C1', ls=':')
    axs[0].set_title(f'exp {k}')
    
    axs[1].plot(distance_idx, pos_matrix[0], label='x')
    axs[1].plot(distance_idx, pos_matrix[1], label='y')
    axs[1].legend()